In [1]:
using HorizonSideRobots

In [2]:
a = Robot("untitled.sit", animate=true);

┌ Warning: Found `resolution` in the theme when creating a `Scene`. The `resolution` keyword for `Scene`s and `Figure`s has been deprecated. Use `Figure(; size = ...` or `Scene(; size = ...)` instead, which better reflects that this is a unitless size and not a pixel resolution. The key could also come from `set_theme!` calls or related theming functions.
└ @ Makie C:\Users\Alex\.julia\packages\Makie\Qvk4f\src\scenes.jl:220


In [3]:
function inverse(side)
    if side == Ost
        return West
    end
    if side == West
        return Ost
    end
    if side == Sud
        return Nord
    end
    if side == Nord
        return Sud
    end
end

inverse (generic function with 1 method)

In [4]:
function along!(r, side)
    while !isborder(r, side)
        move!(r, side)
    end
end

along! (generic function with 1 method)

In [5]:
function try_move!(r, side)
    if !isborder(r, side)
        move!(r, side)
        return true
    end
    return false
end

try_move! (generic function with 1 method)

In [6]:
function HorizonSideRobots.move!(r, side, n_steps)
    steps_done = 0
    for _ in range(1, n_steps)
        if try_move!(r, side)
            steps_done += 1
        end
    end
    return steps_done
end

In [7]:
function snake_chess!(r, side, ort_side, prev_marked)
    move_side = side
    steps_ort = 1
    prev_marked_next = mark_any_line!(r, side, prev_marked)
    while !isborder(r, ort_side)
        move!(r, ort_side)
        steps_ort += 1
        move_side = inverse(move_side)
        prev_marked_next = mark_any_line!(r, move_side, prev_marked_next)
    end
    return steps_ort, move_side == ort_side
end

snake_chess! (generic function with 1 method)

In [8]:
function mark_any_line!(r, move_side, prev_marked)
    if prev_marked == false
        putmarker!(r)
    end
    while !isborder(r, move_side)
        steps_done = move!(r, move_side, 2)
        if steps_done == 2
            putmarker!(r)
        end
    end
    if ismarker(r)
        return true
    end
    return false
end

mark_any_line! (generic function with 1 method)

In [9]:
function mark_first_line!(r)
    putmarker!(r)
    steps_ost = 0
    while !isborder(r, Ost)
        steps_done = move!(r, Ost, 2)
        if steps_done == 2
            putmarker!(r)
        end
        steps_ost += steps_done
    end
    move!(r, West, steps_ost)
    steps_west = 0
    while !isborder(r, West)
        steps_done = move!(r, West, 2)
        if steps_done == 2
            putmarker!(r)
        end
        steps_west += steps_done
    end
    return steps_west
end

mark_first_line! (generic function with 1 method)

In [10]:
function main!(r)
    steps_west = mark_first_line!(r)
    first_marked = ismarker(r)
    move!(r, Sud)
    steps_sud, side_back = snake_chess!(a, Ost, Sud, first_marked)
    move!(a, Nord, steps_sud)
    if side_back
        along!(a, Ost)
    else
        along!(a, West)
    end
    first_marked = ismarker(r)
    move!(r, Nord)
    steps_nord, side_back = snake_chess!(r, Ost, Nord, first_marked)
    move!(a, Sud, steps_nord)
    if side_back
        along!(a, Ost)
    else
        along!(a, West)
    end
    move!(r, Ost, steps_west)
end

main! (generic function with 1 method)

In [11]:
main!(a)

ErrorException: Робот врезался в перегородку при попытке сделать шаг в направлении Sud